In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from functools import reduce
from scipy.stats import binomtest
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize

In [2]:
# # load the input df
# boolean_input_df = pd.read_csv("/data6/deepro/computational_pipelines/pyrarecomb/test/input/test_input.csv")
# # define all other params
# combo_length = 2
# min_indv_threshold = 5
# max_freq_threshold = 0.25
# input_format = 'Input_'
# output_format = 'Output_'
# pval_filter_threshold = 0.05
# adj_pval_type = 'BH'
# min_power_threshold = 0.7
# sample_names_ind = 'Y'

# load the input df
boolean_input_df = pd.read_csv("/data6/deepro/computational_pipelines/pyrarecomb/test/input/test_input.csv")
primary_inputs = pd.read_csv("/data6/deepro/computational_pipelines/pyrarecomb/test/input/primary.txt", header=None).iloc[:, 0].values
# define all other params
combo_length = 2
min_indv_threshold = 5
max_freq_threshold = 0.25
input_format = 'Input_'
output_format = 'Output_'
pval_filter_threshold = 0.05
adj_pval_type = 'BH'
min_power_threshold = 0.7
sample_names_ind = 'Y'
primary_input_entities= primary_inputs



In [6]:
##########
# Filter #
##########

def preprocess_boolean(boolean_input_df, input_format, output_format, min_indv_threshold, max_freq_threshold):
    """
    This function parses and filters user-given boolean dataframe to rarecomb
    1) It defines input and output columns
    2) It creates case and control boolean matrix for data mining algorithm
    3) It filters the matrix to only keep relevant items based on user defined conditions
    """
    # Identify all the input and output variables
    input_colname_list = [col for col in boolean_input_df.columns if col.startswith(input_format)]
    output_column = [col for col in boolean_input_df.columns if col.startswith(output_format)][0]
    # Make cases and controls apriori input df
    apriori_input_cases_df = boolean_input_df.loc[boolean_input_df[output_column]==1, input_colname_list].applymap(int)
    apriori_input_controls_df = boolean_input_df.loc[boolean_input_df[output_column]==0, input_colname_list].applymap(int)
    # Get the number of cases from input param - max freq thresh
    number_of_cases = apriori_input_cases_df.shape[0]
    max_instances = round(number_of_cases * max_freq_threshold)
    # Filter case and control df to only include gene cols that satisfy input criterion
    apriori_input_cases_df = apriori_input_cases_df.loc[:, (apriori_input_cases_df.sum() >= min_indv_threshold) 
                                                        & (apriori_input_controls_df.sum() >= 1) & (apriori_input_cases_df.sum() < max_instances)]
    apriori_input_controls_df = apriori_input_controls_df.loc[:, (apriori_input_cases_df.sum() >= min_indv_threshold) 
                                                              & (apriori_input_controls_df.sum() >= 1) & (apriori_input_cases_df.sum() < max_instances)]
    # Select columns that remain
    sel_input_colname_list = [col for col in apriori_input_cases_df.columns if col.startswith(input_format)]
    return apriori_input_cases_df, apriori_input_controls_df, sel_input_colname_list, output_column, number_of_cases 


apriori_input_cases_df, apriori_input_controls_df, sel_input_colname_list, output_column, number_of_cases = preprocess_boolean(
    boolean_input_df, input_format, output_format, min_indv_threshold, max_freq_threshold
)
input_colname_only_list = list(set(sel_input_colname_list).difference(set(primary_input_entities)))
primary_input_list = list(set(sel_input_colname_list).intersection(set(primary_input_entities)))
# debugging
print(f"Number of cases remaining after filtration: {len(apriori_input_cases_df)}")
print(f"Number of controls remaining after filtration: {len(apriori_input_controls_df)}")
print(f"Number of items remaining after filtration: {len(sel_input_colname_list)}")
print(f"Number of secondary items remaining after filtration: {len(input_colname_only_list)}")
print(f"Number of primary items remaining after filtration: {len(primary_input_list)}")

Number of cases remaining after filtration: 2488
Number of controls remaining after filtration: 2512
Number of items remaining after filtration: 334
Number of secondary items remaining after filtration: 299
Number of primary items remaining after filtration: 35


In [64]:
def get_freq_items_combo(frequent_itemsets, combo_length):
    # get items of specific combo size
    frequent_itemsets = frequent_itemsets.loc[frequent_itemsets.length==combo_length].drop(columns=["length"])
    # Split itemsets into separate columns
    frequent_itemsets = frequent_itemsets.merge(frequent_itemsets['itemsets'].apply(lambda x: pd.Series(list(x))), left_index=True, right_index=True).drop(columns=["support", "itemsets"])
    # rename columns
    old_colnames = range(combo_length)
    new_colnames = [f"Item_{i}" for i in range(1, combo_length+1)]
    frequent_itemsets = frequent_itemsets.rename(columns=dict(zip(old_colnames, new_colnames)))
    return frequent_itemsets.loc[:, new_colnames + ["uniq_items", "Obs_Count_Combo"]].reset_index(drop=True)

def add_frozensets(a, b):
    return a.union(b)  

def run_apriori_freqitems(apriori_input_df, combo_length, support_threshold, primary_entities=None):
    frequent_itemsets = apriori(
        apriori_input_df.astype(bool), min_support=support_threshold, 
        use_colnames=True, max_len=combo_length
        )
    if primary_entities is not None:
        # run association rules
        assoc_df = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.0)
        # the primary entities must be in the consequents only
        assoc_df = assoc_df.loc[
            (assoc_df.consequents.apply(lambda x: True if len(x.intersection(primary_entities))>0 else False)) &
            (assoc_df.antecedents.apply(lambda x: True if len(x.intersection(primary_entities))==0 else False))
            ]
        # convert assoc df to frequent items
        assoc_df["itemsets"] = assoc_df.apply(lambda row: row["antecedents"].union(row["consequents"]), axis=1)
        # get the individual items from frequent items
        frequent_itemsets = frequent_itemsets.loc[frequent_itemsets.itemsets.apply(lambda x: len(x)==1)]
        # add the filtered items obtained from association mining
        frequent_itemsets = pd.concat((frequent_itemsets, assoc_df.loc[:, ["itemsets", "support"]]))

    frequent_itemsets['count'] = frequent_itemsets['support'] * len(apriori_input_df)
    frequent_itemsets['Obs_Count_Combo'] = frequent_itemsets.pop('count')
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets['uniq_items'] = frequent_itemsets['itemsets'].apply(lambda x: "|".join(sorted(x)))
    frequent_itemsets_len_combo = get_freq_items_combo(frequent_itemsets, combo_length)
    frequent_itemsets_len_1 = get_freq_items_combo(frequent_itemsets, 1)
    return frequent_itemsets_len_combo, frequent_itemsets_len_1


In [65]:
############################
# CASES / SEVERE Phenotype #
############################################################################################################
# APRIORI (Combo): Generate frequent itemset of a given size in which mutations/events co-occur among them #
############################################################################################################
# Introduce a support threshold
support_threshold = min_indv_threshold / apriori_input_cases_df.shape[0]
case_freqitems_df, case_freqitems_size1_df = run_apriori_freqitems(apriori_input_cases_df, combo_length, support_threshold, primary_entities=primary_input_list)
# set the number of frequent items column name 
case_freqitems_df = case_freqitems_df.rename(columns={"Obs_Count_Combo": "Case_Obs_Count_Combo"})
# get the number of unique items forming combinations
uniq_combo_items = set(case_freqitems_df.loc[:, [f"Item_{i}" for i in range(1, combo_length+1)]].values.flatten())
# Store the counts as a dictionary for each item
case_freqitems_countdict = dict(zip(case_freqitems_size1_df.Item_1, case_freqitems_size1_df.Obs_Count_Combo.astype(int)))
# Get the observed count for each item
for i in range(1, combo_length+1):
    case_freqitems_df[f"Case_Obs_Count_I{i}"] = case_freqitems_df[f"Item_{i}"].map(case_freqitems_countdict)
# Get the expected probability of observing the combos
case_freqitems_df["Case_Exp_Prob_Combo"] = case_freqitems_df.loc[:, [f"Case_Obs_Count_I{i}" for i in range(1, combo_length+1)]].prod(axis=1)/(number_of_cases**combo_length)
# Get the observed probability of observing the combos
case_freqitems_df['Case_Obs_Prob_Combo'] = case_freqitems_df['Case_Obs_Count_Combo'] / number_of_cases
# Using bionomial test, calculate p-value
case_freqitems_df['Case_pvalue_more'] = case_freqitems_df.apply(lambda row: binomtest(int(row['Case_Obs_Count_Combo']), number_of_cases, row['Case_Exp_Prob_Combo'], alternative='greater').pvalue, axis=1)
# debugging
print(f'Number of initial combinations identified for cases: {case_freqitems_df.shape[0]}')
print(f'Number of unique items in cases: {len(uniq_combo_items)}')

Number of initial combinations identified for cases: 24
Number of unique items in cases: 28


In [66]:
#############################
# CONTROLS / MILD Phenotype #
#############################
# Get the control profile of the combo items
apriori_input_controls_df = apriori_input_controls_df.loc[:, list(uniq_combo_items)]
number_of_controls = apriori_input_controls_df.shape[0]
# define support threshold for controls
support_threshold = 2 / number_of_controls
# get the frequently mutated genes in controls using apriori
sel_primary_input_list = uniq_combo_items.intersection(primary_input_list)
cont_freqitems_df, cont_freqitems_size1_df = run_apriori_freqitems(apriori_input_controls_df, combo_length, support_threshold, primary_entities=sel_primary_input_list)
# set the number of frequent items column name 
cont_freqitems_df = cont_freqitems_df.rename(columns={"Obs_Count_Combo": "Cont_Obs_Count_Combo"})
# Store the counts as a dictionary for each item
cont_freqitems_countdict = dict(zip(cont_freqitems_size1_df.Item_1, cont_freqitems_size1_df.Obs_Count_Combo.astype(int)))
# Keep combos found in case only
case_cont_freqitems_df = case_freqitems_df.merge(cont_freqitems_df, left_on="uniq_items", right_on="uniq_items", how="left", suffixes=('', '_cont')).drop(columns=[f"Item_{i}_cont" for i in range(1, combo_length + 1)]).fillna(0.)
# Get the observed count in controls for each item
for i in range(1, combo_length+1):
    case_cont_freqitems_df[f"Cont_Obs_Count_I{i}"] = case_cont_freqitems_df[f"Item_{i}"].map(cont_freqitems_countdict)
# Get the expected probability of observing the combos in controls
case_cont_freqitems_df["Cont_Exp_Prob_Combo"] = case_cont_freqitems_df.loc[:, [f"Cont_Obs_Count_I{i}" for i in range(1, combo_length+1)]].prod(axis=1)/(number_of_controls**combo_length)
# Get the observed probability of observing the combos
case_cont_freqitems_df['Cont_Obs_Prob_Combo'] = case_cont_freqitems_df['Cont_Obs_Count_Combo'] / number_of_controls
# Using bionomial test, calculate p-value
case_cont_freqitems_df['Cont_pvalue_more'] = case_cont_freqitems_df.apply(lambda row: binomtest(int(row['Cont_Obs_Count_Combo']), number_of_controls, row['Cont_Exp_Prob_Combo'], alternative='greater').pvalue, axis=1)
# debugging
print(f"Number of controls: {number_of_controls}")
print(f"Number of combinations with support of at least 2 in controls: {cont_freqitems_df.shape[0]}")

Number of controls: 2512
Number of combinations with support of at least 2 in controls: 92


In [77]:
########################
# Nominal significance #
########################
# TODO: This step is not omitted from compare_enrichment_depletion since we
# TODO: consider all for multiple testing.
sel_case_cont_freqitems_df = case_cont_freqitems_df
# debugging
print(f"Number of combinations considered for multiple testing correction: {sel_case_cont_freqitems_df.shape[0]}")

####################
# Multiple testing #
####################
# Create variable for number of tests done
number_of_tests = sel_case_cont_freqitems_df.shape[0]
# multiple test BH and Bonferroni
sel_case_cont_freqitems_df['Case_Adj_Pval_bonf'] = np.round(multipletests(sel_case_cont_freqitems_df['Case_pvalue_more'].values, method='bonferroni')[1], 3)
sel_case_cont_freqitems_df['Case_Adj_Pval_BH'] = np.round(multipletests(sel_case_cont_freqitems_df['Case_pvalue_more'].values, method='fdr_bh')[1], 3)
# add a column for number of tests done
sel_case_cont_freqitems_df['Num_tests'] = number_of_tests
# filter significant items
if adj_pval_type == 'BH':
    all_sig_case_cont_freqitems_df = sel_case_cont_freqitems_df[
        (sel_case_cont_freqitems_df['Case_Adj_Pval_BH'] < pval_filter_threshold) &
        (sel_case_cont_freqitems_df['Cont_pvalue_more'] > pval_filter_threshold)
    ]
elif adj_pval_type == 'bonferroni':
    all_sig_case_cont_freqitems_df = sel_case_cont_freqitems_df[
        (sel_case_cont_freqitems_df['Case_Adj_Pval_bonf'] < pval_filter_threshold) &
        (sel_case_cont_freqitems_df['Cont_pvalue_more'] > pval_filter_threshold)
    ]
multtest_sig_comb_count = all_sig_case_cont_freqitems_df.shape[0]
# debugging
print(f"Number of combinations that are significant after multiple testing correction: {multtest_sig_comb_count}")


Number of combinations considered for multiple testing correction: 24
Number of combinations that are significant after multiple testing correction: 4


In [79]:

def get_counts(uniq_items, input_df):
    """
    This function gets the counts of combos from an input boolean df
    """
    query = " & ".join([f"({i} == 1)" for i in uniq_items.split("|")])
    return len(input_df.query(query))

def refine_control_frequencies(all_sig_case_cont_freqitems_df, apriori_input_controls_df,number_of_controls,check_enrichment=True):
    # Check if zero frequency cases exist in controls
    cont_combos_w_zero_freq_df = all_sig_case_cont_freqitems_df.loc[all_sig_case_cont_freqitems_df["Cont_Obs_Count_Combo"] == 0]
    zero_freq_combo_count = cont_combos_w_zero_freq_df.shape[0]
    print('Number of combinations with support less than 2 in controls:', zero_freq_combo_count)
    if zero_freq_combo_count > 0:
        # REFINE CONTROL FREQUENCIES
        # for the zero frequency combos in controls, get their actual combo size
        cont_combos_w_zero_freq_df["Cont_Obs_Count_Combo"] = cont_combos_w_zero_freq_df.uniq_items.apply(get_counts, args=(apriori_input_controls_df, ))
        cont_combos_w_zero_freq_df["Cont_Obs_Prob_Combo"] = cont_combos_w_zero_freq_df['Cont_Obs_Count_Combo'] / number_of_controls
        if check_enrichment:
            cont_combos_w_zero_freq_df['Cont_pvalue_more'] = cont_combos_w_zero_freq_df.apply(lambda row: binomtest(int(row['Cont_Obs_Count_Combo']), number_of_controls, row['Cont_Exp_Prob_Combo'], alternative='greater').pvalue, axis=1)
        else:
            # check for depletion
            cont_combos_w_zero_freq_df['Cont_pvalue_less'] = cont_combos_w_zero_freq_df.apply(lambda row: binomtest(int(row['Cont_Obs_Count_Combo']), number_of_controls, row['Cont_Exp_Prob_Combo'], alternative='less').pvalue, axis=1)
        # rewrite the items in the main df
        all_sig_case_cont_freqitems_df.update(cont_combos_w_zero_freq_df)
    return all_sig_case_cont_freqitems_df

def calculate_power(all_sig_case_cont_freqitems_df, number_of_cases, number_of_controls):
    all_sig_case_cont_freqitems_df['Case_Exp_Count_Combo'] = round((all_sig_case_cont_freqitems_df['Case_Exp_Prob_Combo'] * number_of_cases), 2)
    all_sig_case_cont_freqitems_df['Cont_Exp_Count_Combo'] = round((all_sig_case_cont_freqitems_df['Cont_Exp_Prob_Combo'] * number_of_controls), 2)
    all_sig_case_cont_freqitems_df['Effect_Size'] = all_sig_case_cont_freqitems_df.apply(
        lambda row: proportion_effectsize(row['Case_Obs_Prob_Combo'], row['Cont_Obs_Prob_Combo']), axis=1)
    all_sig_case_cont_freqitems_df['Power_One_Pct'] = round(all_sig_case_cont_freqitems_df.apply(
        lambda row: tt_ind_solve_power(effect_size=row['Effect_Size'], nobs1=number_of_cases, ratio=number_of_controls/number_of_cases, alpha=0.01), axis=1), 3)
    all_sig_case_cont_freqitems_df['Power_Five_Pct'] = round(all_sig_case_cont_freqitems_df.apply(
        lambda row: tt_ind_solve_power(effect_size=row['Effect_Size'], nobs1=number_of_cases, ratio=number_of_controls/number_of_cases, alpha=0.05),axis=1), 3)
    return all_sig_case_cont_freqitems_df

def get_samples(row, samples_df, output_column):
    """
    Helper function for adding sample information
    """
    items = row.uniq_items.split("|")
    count_cases = Counter(samples_df.loc[(samples_df.Items.isin(items))&(samples_df[output_column]==1)].Sample_Name)
    case_samples = [s for s,ns in count_cases.items() if ns==len(items)]
    count_controls = Counter(samples_df.loc[(samples_df.Items.isin(items))&(samples_df[output_column]==0)].Sample_Name)
    control_samples = [s for s,ns in count_controls.items() if ns==len(items)]
    return pd.Series({"Case_Samples": "|".join(case_samples), "Control_Samples": "|".join(control_samples)})

def add_sample_info(boolean_input_df, output_sig_case_cont_freqitems_df, output_column):
    # add case and control samples for each combo
    samples_df = boolean_input_df.set_index(["Sample_Name", output_column])
    samples_df = samples_df.mask(samples_df == 0).stack().reset_index().drop(0, axis=1).rename(columns={"level_2": "Items"})
    output_sig_case_cont_freqitems_df = output_sig_case_cont_freqitems_df.merge(output_sig_case_cont_freqitems_df.apply(get_samples, args=(samples_df, output_column), axis=1), left_index=True, right_index=True)
    return output_sig_case_cont_freqitems_df


In [80]:
###################
# Post processing #
###################
# Check if there is at least a single significant combination after multiple testing correction
if multtest_sig_comb_count > 0:
    # TODO: Refining control frequencies may not be required if support is changed to 1/num_controls for combos
    all_sig_case_cont_freqitems_df = refine_control_frequencies(all_sig_case_cont_freqitems_df, apriori_input_controls_df,number_of_controls)

    ######################
    # POWER CALCULATIONS #
    ######################
    all_sig_case_cont_freqitems_df = calculate_power(all_sig_case_cont_freqitems_df, number_of_cases, number_of_controls)
    output_sig_case_cont_freqitems_df = all_sig_case_cont_freqitems_df.loc[all_sig_case_cont_freqitems_df.Power_Five_Pct >= min_power_threshold]
    
    #####################
    # SAMPLES DETECTION #
    #####################
    if len(output_sig_case_cont_freqitems_df)>0:
        print(f"Number of significant combinations that meet the power threshold is {len(output_sig_case_cont_freqitems_df)}")
        if sample_names_ind == "Y":
            output_sig_case_cont_freqitems_df = add_sample_info(boolean_input_df, output_sig_case_cont_freqitems_df, output_column)

    
    else:
        print("No significant combinations that meet the specified power threshold")
        print("Returning ONLY the non-significant combinations")
        output_sig_case_cont_freqitems_df = sel_case_cont_freqitems_df

else:
    print("No significant combinations were found after multiple testing correction")
    print("Returning ONLY the non-significant combinations")
    output_sig_case_cont_freqitems_df = sel_case_cont_freqitems_df

Number of combinations with support less than 2 in controls: 1
Number of significant combinations that meet the power threshold is 1


/tmp/ipykernel_14193/2212866994.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cont_combos_w_zero_freq_df["Cont_Obs_Count_Combo"] = cont_combos_w_zero_freq_df.uniq_items.apply(get_counts, args=(apriori_input_controls_df, ))
/tmp/ipykernel_14193/2212866994.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cont_combos_w_zero_freq_df["Cont_Obs_Prob_Combo"] = cont_combos_w_zero_freq_df['Cont_Obs_Count_Combo'] / number_of_controls
/tmp/ipykernel_14193/2212866994.py:19: SettingWithCopyWarning: 
A value i

In [82]:
all_sig_case_cont_freqitems_df

,Item_1,Item_2,uniq_items,Case_Obs_Count_Combo,Case_Obs_Count_I1,Case_Obs_Count_I2,Case_Exp_Prob_Combo,Case_Obs_Prob_Combo,Case_pvalue_more,Cont_Obs_Count_Combo,...,Cont_Obs_Prob_Combo,Cont_pvalue_more,Case_Adj_Pval_bonf,Case_Adj_Pval_BH,Num_tests,Case_Exp_Count_Combo,Cont_Exp_Count_Combo,Effect_Size,Power_One_Pct,Power_Five_Pct
0,Input_7,Input_65,Input_65|Input_7,6.0,28,80,0.000362,0.002412,3.426543e-04,3.0,...,0.001194,0.119919,0.008,0.003,24,0.90,1.20,0.029125,0.061,0.177
4,Input_46,Input_158,Input_158|Input_46,5.0,47,36,0.000273,0.002010,6.890116e-04,2.0,...,0.000796,0.170247,0.017,0.004,24,0.68,0.74,0.033247,0.081,0.217
5,Input_353,Input_55,Input_353|Input_55,5.0,14,19,0.000043,0.002010,1.060869e-07,0.0,...,0.000000,1.000000,0.000,0.000,24,0.11,0.07,0.089688,0.724,0.887
22,Input_445,Input_462,Input_445|Input_462,7.0,74,73,0.000873,0.002814,6.943866e-03,4.0,...,0.001592,0.082795,0.167,0.028,24,2.17,1.63,0.026305,0.050,0.153


In [81]:
output_sig_case_cont_freqitems_df

,Item_1,Item_2,uniq_items,Case_Obs_Count_Combo,Case_Obs_Count_I1,Case_Obs_Count_I2,Case_Exp_Prob_Combo,Case_Obs_Prob_Combo,Case_pvalue_more,Cont_Obs_Count_Combo,...,Case_Adj_Pval_bonf,Case_Adj_Pval_BH,Num_tests,Case_Exp_Count_Combo,Cont_Exp_Count_Combo,Effect_Size,Power_One_Pct,Power_Five_Pct,Case_Samples,Control_Samples
5,Input_353,Input_55,Input_353|Input_55,5.0,14,19,0.000043,0.00201,1.060869e-07,0.0,...,0.0,0.0,24,0.11,0.07,0.089688,0.724,0.887,Sample_1425|Sample_2118|Sample_2284|Sample_403...,


In [33]:
##################################################################################
# APRIORI (Individual): Generate frequencies of event for each individual entity #
##################################################################################
support_threshold = min_indv_threshold / apriori_input_cases_df.shape[0]
include_output_ind = "N"
# get the frequency of mutations for individual genes using apriori
case_freqitems_size1_df = run_apriori_freqitems(apriori_input_cases_df, 1, support_threshold, uniq_combo_items, include_output_ind=include_output_ind)
# Store the counts as a dictionary for each item
case_freqitems_countdict = dict(zip(case_freqitems_size1_df.Item_1, case_freqitems_size1_df.Obs_Count_Combo.astype(int)))
# Get the observed count for each item
for i in range(1, combo_length+1):
    case_freqitems_df[f"Case_Obs_Count_I{i}"] = case_freqitems_df[f"Item_{i}"].map(case_freqitems_countdict)
# Get the expected probability of observing the combos
case_freqitems_df["Case_Exp_Prob_Combo"] = case_freqitems_df.loc[:, [f"Case_Obs_Count_I{i}" for i in range(1, combo_length+1)]].prod(axis=1)/(number_of_cases**combo_length)
# Get the observed probability of observing the combos
case_freqitems_df['Case_Obs_Prob_Combo'] = case_freqitems_df['Case_Obs_Count_Combo'] / number_of_cases
# Using bionomial test, calculate p-value
case_freqitems_df['Case_pvalue_more'] = case_freqitems_df.apply(lambda row: binomtest(int(row['Case_Obs_Count_Combo']), number_of_cases, row['Case_Exp_Prob_Combo'], alternative='greater').pvalue, axis=1)


In [34]:
#############################
# CONTROLS / MILD Phenotype #
############################################################################################################
# APRIORI (Combo): Generate frequent itemset of a given size in which mutations/events co-occur among them #
############################################################################################################
# Get the control profile of the combo items
apriori_input_controls_df = apriori_input_controls_df.loc[:, list(uniq_combo_items)]
number_of_controls = apriori_input_controls_df.shape[0]
print(f"Number of controls: {number_of_controls}")

# define support threshold for controls
support_threshold = 2 / number_of_controls
include_output_ind = "N"

# get the frequently mutated genes in controls using apriori
cont_freqitems_df = run_apriori_freqitems(apriori_input_controls_df, combo_length, support_threshold, uniq_combo_items, include_output_ind=include_output_ind)
# set the number of frequent items column name 
cont_freqitems_df = cont_freqitems_df.rename(columns={"Obs_Count_Combo": "Cont_Obs_Count_Combo"})
print(f"Number of combinations with support of at least 2 in controls: {cont_freqitems_df.shape[0]}")

Number of controls: 2512


Number of combinations with support of at least 2 in controls: 1357


In [35]:
##################################################################################
# APRIORI (Individual): Generate frequencies of event for each individual entity #
##################################################################################

support_threshold = 1 / apriori_input_controls_df.shape[0]
include_output_ind = "N"

cont_freqitems_size1_df = run_apriori_freqitems(apriori_input_controls_df, 1, support_threshold, uniq_combo_items, include_output_ind=include_output_ind)
# Store the counts as a dictionary for each item
cont_freqitems_countdict = dict(zip(cont_freqitems_size1_df.Item_1, cont_freqitems_size1_df.Obs_Count_Combo.astype(int)))
# Keep combos found in case only
case_freqitems_df["uniq_items"] = case_freqitems_df.loc[:, [f"Item_{i}" for i in range(1, combo_length + 1)]].apply(lambda x: "|".join(sorted(x)), axis=1)
cont_freqitems_df["uniq_items"] = cont_freqitems_df.loc[:, [f"Item_{i}" for i in range(1, combo_length + 1)]].apply(lambda x: "|".join(sorted(x)), axis=1)
case_cont_freqitems_df = case_freqitems_df.merge(cont_freqitems_df, left_on="uniq_items", right_on="uniq_items", how="left", suffixes=('', '_cont')).drop(columns=[f"Item_{i}_cont" for i in range(1, combo_length + 1)]).fillna(0.)
# Get the observed count in controls for each item
for i in range(1, combo_length+1):
    case_cont_freqitems_df[f"Cont_Obs_Count_I{i}"] = case_cont_freqitems_df[f"Item_{i}"].map(cont_freqitems_countdict)
# Get the expected probability of observing the combos in controls
case_cont_freqitems_df["Cont_Exp_Prob_Combo"] = case_cont_freqitems_df.loc[:, [f"Cont_Obs_Count_I{i}" for i in range(1, combo_length+1)]].prod(axis=1)/(number_of_controls**combo_length)
# Get the observed probability of observing the combos
case_cont_freqitems_df['Cont_Obs_Prob_Combo'] = case_cont_freqitems_df['Cont_Obs_Count_Combo'] / number_of_controls
# Using bionomial test, calculate p-value
case_cont_freqitems_df['Cont_pvalue_more'] = case_cont_freqitems_df.apply(lambda row: binomtest(int(row['Cont_Obs_Count_Combo']), number_of_controls, row['Cont_Exp_Prob_Combo'], alternative='greater').pvalue, axis=1)
# get cases where case pvalue and control pvalue are both significant
filt_case_cont_freqitems_df = case_cont_freqitems_df.loc[(case_cont_freqitems_df['Case_pvalue_more'] < pval_filter_threshold) &
                                                        (case_cont_freqitems_df['Cont_pvalue_more'] < pval_filter_threshold)
                                                        ]
# get cases where either case pvalue or control pvalue are not significant                   
sel_case_cont_freqitems_df = case_cont_freqitems_df.loc[~((case_cont_freqitems_df['Case_pvalue_more'] < pval_filter_threshold) &
                                                        (case_cont_freqitems_df['Cont_pvalue_more'] < pval_filter_threshold))
                                                        ]
# debugging
print(f"Number of combinations that are enriched in both cases and controls: {filt_case_cont_freqitems_df.shape[0]}")
print(f"Number of combinations considered for multiple testing correction: {sel_case_cont_freqitems_df.shape[0]}")

Number of combinations that are enriched in both cases and controls: 47
Number of combinations considered for multiple testing correction: 246


In [36]:
####################
# Multiple testing #
####################

# Create variable for number of tests done
number_of_tests = sel_case_cont_freqitems_df.shape[0]

sel_case_cont_freqitems_df['Case_Adj_Pval_bonf'] = multipletests(sel_case_cont_freqitems_df['Case_pvalue_more'].values, method='bonferroni')[1]
sel_case_cont_freqitems_df['Case_Adj_Pval_BH'] = multipletests(sel_case_cont_freqitems_df['Case_pvalue_more'].values, method='fdr_bh')[1]

if adj_pval_type == 'BH':
    all_sig_case_cont_freqitems_df = sel_case_cont_freqitems_df[
        (sel_case_cont_freqitems_df['Case_Adj_Pval_BH'] < pval_filter_threshold) &
        (sel_case_cont_freqitems_df['Cont_pvalue_more'] > pval_filter_threshold)
    ]
elif adj_pval_type == 'bonferroni':
    all_sig_case_cont_freqitems_df = sel_case_cont_freqitems_df[
        (sel_case_cont_freqitems_df['Case_Adj_Pval_bonf'] < pval_filter_threshold) &
        (sel_case_cont_freqitems_df['Cont_pvalue_more'] > pval_filter_threshold)
    ]

multtest_sig_comb_count = all_sig_case_cont_freqitems_df.shape[0]
# debugging
print(f"Number of combinations that are significant after multiple testing correction: {multtest_sig_comb_count}")

Number of combinations that are significant after multiple testing correction: 91


/tmp/ipykernel_89739/86612666.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_case_cont_freqitems_df['Case_Adj_Pval_bonf'] = multipletests(sel_case_cont_freqitems_df['Case_pvalue_more'].values, method='bonferroni')[1]
/tmp/ipykernel_89739/86612666.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_case_cont_freqitems_df['Case_Adj_Pval_BH'] = multipletests(sel_case_cont_freqitems_df['Case_pvalue_more'].values, method='fdr_bh')[1]


In [37]:
def get_counts(uniq_items, input_df):
    query = " & ".join([f"({i} == 1)" for i in uniq_items.split("|")])
    return len(input_df.query(query))

def get_samples(row, samples_df, output_column):
    items = row.uniq_items.split("|")
    count_cases = Counter(samples_df.loc[(samples_df.Items.isin(items))&(samples_df[output_column]==1)].Sample_Name)
    case_samples = [s for s,ns in count_cases.items() if ns==len(items)]
    count_controls = Counter(samples_df.loc[(samples_df.Items.isin(items))&(samples_df[output_column]==0)].Sample_Name)
    control_samples = [s for s,ns in count_controls.items() if ns==len(items)]
    return pd.Series({"Case_Samples": "|".join(case_samples), "Control_Samples": "|".join(control_samples)})

In [38]:
###################
# Post processing #
###################

# Check if there is at least a single significant combination after multiple testing correction
if multtest_sig_comb_count > 0:
    # Check if zero frequency cases exist in controls
    cont_combos_w_zero_freq_df = all_sig_case_cont_freqitems_df.loc[all_sig_case_cont_freqitems_df["Cont_Obs_Count_Combo"] == 0]
    zero_freq_combo_count = cont_combos_w_zero_freq_df.shape[0]
    print('Number of combinations with support less than 2 in controls:', zero_freq_combo_count)

    if zero_freq_combo_count > 0:
        # REFINE CONTROL FREQUENCIES
        # for the zero frequency combos in controls, get their actual combo size
        cont_combos_w_zero_freq_df["Cont_Obs_Count_Combo"] = cont_combos_w_zero_freq_df.uniq_items.apply(get_counts, args=(apriori_input_controls_df, ))
        cont_combos_w_zero_freq_df["Cont_Obs_Prob_Combo"] = cont_combos_w_zero_freq_df['Cont_Obs_Count_Combo'] / number_of_controls
        cont_combos_w_zero_freq_df['Cont_pvalue_more'] = cont_combos_w_zero_freq_df.apply(lambda row: binomtest(int(row['Cont_Obs_Count_Combo']), number_of_controls, row['Cont_Exp_Prob_Combo'], alternative='greater').pvalue, axis=1)
        # rewrite the items in the main df
        all_sig_case_cont_freqitems_df.update(cont_combos_w_zero_freq_df)
    else:
        print("there are no combinations with zero frequency count in controls")
    
    ######################
    # POWER CALCULATIONS #
    ######################
    all_sig_case_cont_freqitems_df['Case_Exp_Count_Combo'] = round((all_sig_case_cont_freqitems_df['Case_Exp_Prob_Combo'] * number_of_cases), 2)
    all_sig_case_cont_freqitems_df['Cont_Exp_Count_Combo'] = round((all_sig_case_cont_freqitems_df['Cont_Exp_Prob_Combo'] * number_of_controls), 2)


    all_sig_case_cont_freqitems_df['Effect_Size'] = all_sig_case_cont_freqitems_df.apply(
        lambda row: proportion_effectsize(row['Case_Obs_Prob_Combo'], row['Cont_Obs_Prob_Combo']),
        axis=1
    )

    all_sig_case_cont_freqitems_df['Power_One_Pct'] = round(all_sig_case_cont_freqitems_df.apply(
        lambda row: tt_ind_solve_power(effect_size=row['Effect_Size'], nobs1=number_of_cases, ratio=number_of_controls/number_of_cases, alpha=0.01),
        axis=1
    ), 3)

    all_sig_case_cont_freqitems_df['Power_Five_Pct'] = round(all_sig_case_cont_freqitems_df.apply(
        lambda row: tt_ind_solve_power(effect_size=row['Effect_Size'], nobs1=number_of_cases, ratio=number_of_controls/number_of_cases, alpha=0.05),
        axis=1
    ), 3)

    output_sig_case_cont_freqitems_df = all_sig_case_cont_freqitems_df.loc[all_sig_case_cont_freqitems_df.Power_Five_Pct >= min_power_threshold]
    
    if len(output_sig_case_cont_freqitems_df)>0:
        print(f"Number of significant combinations that meet the power threshold is {len(output_sig_case_cont_freqitems_df)}")
        if sample_names_ind == "N":
            output_sig_case_cont_freqitems_df["Num_tests"] = number_of_tests
        else:
            # add case and control samples for each combo
            samples_df = boolean_input_df.set_index(["Sample_Name", output_column])
            samples_df = samples_df.mask(samples_df == 0).stack().reset_index().drop(0, axis=1).rename(columns={"level_2": "Items"})
            output_sig_case_cont_freqitems_df = output_sig_case_cont_freqitems_df.merge(output_sig_case_cont_freqitems_df.apply(get_samples, args=(samples_df, output_column), axis=1), left_index=True, right_index=True)
            output_sig_case_cont_freqitems_df['Num_tests'] = number_of_tests
    
    else:
        print("No significant combinations that meet the specified power threshold")
        print("Returning ONLY the non-significant combinations")
        # add a column for number of tests done
        sel_case_cont_freqitems_df['Num_tests'] = number_of_tests

else:
    print("No significant combinations were found after multiple testing correction")
    print("Returning ONLY the non-significant combinations")
    # add a column for number of tests done
    sel_case_cont_freqitems_df['Num_tests'] = number_of_tests


Number of combinations with support less than 2 in controls: 37


/tmp/ipykernel_89739/293949802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cont_combos_w_zero_freq_df["Cont_Obs_Count_Combo"] = cont_combos_w_zero_freq_df.uniq_items.apply(get_counts, args=(apriori_input_controls_df, ))
/tmp/ipykernel_89739/293949802.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cont_combos_w_zero_freq_df["Cont_Obs_Prob_Combo"] = cont_combos_w_zero_freq_df['Cont_Obs_Count_Combo'] / number_of_controls
/tmp/ipykernel_89739/293949802.py:17: SettingWithCopyWarning: 
A value is t

Number of significant combinations that meet the power threshold is 21


/tmp/ipykernel_89739/293949802.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sig_case_cont_freqitems_df['Power_One_Pct'] = round(all_sig_case_cont_freqitems_df.apply(
/tmp/ipykernel_89739/293949802.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sig_case_cont_freqitems_df['Power_Five_Pct'] = round(all_sig_case_cont_freqitems_df.apply(


In [39]:
output_sig_case_cont_freqitems_df

,Item_1,Item_2,Case_Obs_Count_Combo,Case_Obs_Count_I1,Case_Obs_Count_I2,Case_Exp_Prob_Combo,Case_Obs_Prob_Combo,Case_pvalue_more,uniq_items,Cont_Obs_Count_Combo,...,Case_Adj_Pval_bonf,Case_Adj_Pval_BH,Case_Exp_Count_Combo,Cont_Exp_Count_Combo,Effect_Size,Power_One_Pct,Power_Five_Pct,Case_Samples,Control_Samples,Num_tests
2,Input_9,Input_234,6.0,17,165,0.000453,0.002412,1.092090e-03,Input_234|Input_9,0.0,...,0.268654,0.007892,1.13,1.50,0.098255,0.815,0.935,Sample_2449|Sample_2458|Sample_2862|Sample_382...,,246
7,Input_24,Input_294,5.0,57,19,0.000175,0.002010,9.048240e-05,Input_24|Input_294,0.0,...,0.022259,0.001712,0.44,0.25,0.089688,0.724,0.887,Sample_509|Sample_868|Sample_2186|Sample_2360|...,,246
16,Input_27,Input_269,7.0,34,106,0.000582,0.002814,7.539108e-04,Input_269|Input_27,0.0,...,0.185462,0.006182,1.45,2.25,0.106135,0.880,0.963,Sample_702|Sample_1921|Sample_1963|Sample_2269...,,246
19,Input_29,Input_372,5.0,23,58,0.000216,0.002010,2.361539e-04,Input_29|Input_372,0.0,...,0.058094,0.003417,0.54,0.42,0.089688,0.724,0.887,Sample_7|Sample_370|Sample_399|Sample_2961|Sam...,,246
23,Input_30,Input_342,8.0,49,121,0.000958,0.003215,3.183206e-03,Input_30|Input_342,0.0,...,0.783069,0.016661,2.38,1.42,0.113470,0.924,0.980,Sample_389|Sample_1525|Sample_2734|Sample_3040...,,246
24,Input_348,Input_30,10.0,117,49,0.000926,0.004019,1.445197e-04,Input_30|Input_348,2.0,...,0.035552,0.002539,2.30,1.89,0.070440,0.466,0.702,Sample_247|Sample_868|Sample_1224|Sample_1525|...,Sample_1125|Sample_3073,246
25,Input_30,Input_372,5.0,49,58,0.000459,0.002010,6.327852e-03,Input_30|Input_372,0.0,...,1.000000,0.024323,1.14,0.80,0.089688,0.724,0.887,Sample_1603|Sample_2118|Sample_2734|Sample_449...,,246
30,Input_33,Input_462,5.0,36,73,0.000425,0.002010,4.586176e-03,Input_33|Input_462,0.0,...,1.000000,0.020574,1.06,1.05,0.089688,0.724,0.887,Sample_2118|Sample_2838|Sample_3885|Sample_403...,,246
33,Input_39,Input_300,5.0,35,54,0.000305,0.002010,1.122807e-03,Input_300|Input_39,0.0,...,0.276211,0.007892,0.76,0.52,0.089688,0.724,0.887,Sample_568|Sample_1392|Sample_3244|Sample_3809...,,246
42,Input_55,Input_353,5.0,19,14,0.000043,0.002010,1.060869e-07,Input_353|Input_55,0.0,...,0.000026,0.000013,0.11,0.07,0.089688,0.724,0.887,Sample_1425|Sample_2118|Sample_2284|Sample_403...,,246
